## Note book where I can open a connection to an LLM and give prompts

In [ ]:
# Load Libraries
import osmnx as ox
import networkx as nx
import dash_leaflet as dl
from jupyter_dash import JupyterDash
from dash import html
import dash
import json
from ollama import chat


In [1]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='qwen3:8b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

<think>
Okay, the user asked, "Why is the sky blue?" I need to explain this in a clear and concise way. Let me start by recalling the basic science behind this phenomenon. The sky appears blue due to Rayleigh scattering.

First, I should mention that sunlight is made up of different colors, each with different wavelengths. When sunlight enters Earth's atmosphere, the molecules and small particles in the air scatter the light. Shorter wavelengths, like blue and violet, scatter more than longer wavelengths like red and yellow. 

Wait, but why is the sky blue and not violet? Oh, right, the sun emits more blue light than violet, and our eyes are more sensitive to blue. Also, some of the violet light is absorbed by the upper atmosphere. That's why we see blue instead of violet.

I should also explain that during sunrise or sunset, the sky turns red or orange because the sunlight has to pass through more atmosphere, scattering the blue light out of our line of sight. That's an important poin

In [2]:
response['message']

Message(role='assistant', content='<think>\nOkay, the user asked, "Why is the sky blue?" I need to explain this in a clear and concise way. Let me start by recalling the basic science behind this phenomenon. The sky appears blue due to Rayleigh scattering.\n\nFirst, I should mention that sunlight is made up of different colors, each with different wavelengths. When sunlight enters Earth\'s atmosphere, the molecules and small particles in the air scatter the light. Shorter wavelengths, like blue and violet, scatter more than longer wavelengths like red and yellow. \n\nWait, but why is the sky blue and not violet? Oh, right, the sun emits more blue light than violet, and our eyes are more sensitive to blue. Also, some of the violet light is absorbed by the upper atmosphere. That\'s why we see blue instead of violet.\n\nI should also explain that during sunrise or sunset, the sky turns red or orange because the sunlight has to pass through more atmosphere, scattering the blue light out of

In [3]:
from ollama import chat

stream = chat(
    model='qwen3:8b',#'mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

<think>
Okay, the user is asking why the sky is blue. I need to explain this in a clear and simple way. Let me start by recalling what I know about this. I remember that it has to do with the scattering of sunlight by the atmosphere. 

So, sunlight is white, but it's made up of different colors, each with different wavelengths. When sunlight enters Earth's atmosphere, it interacts with gas molecules and small particles. I think this is called Rayleigh scattering. The shorter wavelengths, like blue and violet, scatter more than the longer ones like red and yellow. 

Wait, but why do we see blue instead of violet? Maybe because the sun emits more blue light than violet, and our eyes are more sensitive to blue. Also, the scattering of violet light is in all directions, but our eyes aren't as sensitive to it, so we perceive the sky as blue. 

I should also mention that during sunrise or sunset, the sky appears red or orange because the sunlight has to pass through more atmosphere, scatteri

KeyboardInterrupt: 

In [ ]:
from ollama import chat

model = 'mistral'

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
  {
    'role': 'assistant',
    'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight.",
  },
  {
    'role': 'user',
    'content': 'What is the weather in Tokyo?',
  },
  {
    'role': 'assistant',
    'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing. The city is known for its high-tech and vibrant culture, with many popular tourist attractions such as the Tokyo Tower, Senso-ji Temple, and the bustling Shibuya district.',
  },
]

#while True:
user_input = input('Chat with history: ')
response = chat(
  model,
  messages=[*messages, {'role': 'user', 'content': user_input}],
)

  # Add the response to the messages to maintain the history
messages += [
  {'role': 'user', 'content': user_input},
  {'role': 'assistant', 'content': response.message.content},
]
print(response.message.content + '\n')

 Sorry, I didn't understand your question. Could you please clarify or rephrase it? I'm here to help! If you have any other questions about travel, weather, or culture in Tokyo, feel free to ask!



In [ ]:
[*messages, {'role': 'user', 'content': user_input}]

[{'role': 'user', 'content': 'Why is the sky blue?'},
 {'role': 'assistant',
  'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight."},
 {'role': 'user', 'content': 'What is the weather in Tokyo?'},
 {'role': 'assistant',
  'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing. The city is known for its high-tech and vibrant culture, with many popular tourist attractions such as the Tokyo Tower, Senso-ji Temple, and the bustling Shibuya district.'},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': " Sorry, I didn't understand your question. Could you please clarify or rephrase it? I'm here to help! If you have any other questions about travel, weather, or culture in Tokyo, feel free to

## Get the nodes
We will get nodes from the area with the test centre

In [ ]:
G = ox.graph_from_point((51.3829463, -0.2933327), dist=5000, network_type='drive')  # Get the OSMNX network of nodes
## Add start nodes
G.add_node('test_centre', x=51.38413, y=-0.29299)
G.add_node('test_centre_road', x=51.38409, y=-0.29280)


In [ ]:
# Useful functions
# Get the nodes on the road
def get_all_nodes_on_street(G, start_node, street_name):
    """This is a 'flood' fill to get all of the nodes on a street

    Args:
        G (_type_): OSMNX network
        start_node (_type_): a starting node on the street
        street_name (_type_): this is the name of the street the current node is on
    """
    visited = set() # empty set to store nodes
    stack = [start_node] # list of nodes
    
    while stack: # while the stack list is occupied
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        
        # Check all outward and inward edges
        for neighbour in G.neighbors(node): # loop through the node neighbours
            for key, data in G.get_edge_data(node, neighbour).items(): # get the edge data
                name = data.get("name") # get the name of the edge
                if (isinstance(name, list) and street_name in name) or name == street_name: # check if the node is on the street
                    if neighbour not in visited:
                        stack.append(neighbour) # append the neigbour to the stack to check
                    
        for pred in G.predecessors(node): # get the predecessor nodes to check for nodes that are on one way roads (may not have upstream edges)
            for key, data in G.get_edge_data(pred, node).items():             
                name = data.get("name")    
                if (isinstance(name, list) and street_name in name or name == street_name):    
                    if pred not in visited: # check the node has already been added to visited
                        stack.append(pred) # add the 
        
    return list(visited)


# Get the edges between nodes

In [ ]:
# Test the getting node functions and plot to see
centre_lat = 51.3829463
centre_lon = -0.2933327

douglas_road_node_ids = get_all_nodes_on_street(G, 23779844, "Douglas Road")
douglas_road_nodes = {node: G.nodes[node] for node in douglas_road_node_ids if node in G.nodes}

douglas_road_markers = [
    dl.Marker(position=(douglas_road_nodes[nid]['y'], douglas_road_nodes[nid]['x']), children=dl.Tooltip(str(nid)))
    for nid in douglas_road_nodes
]

dennan_road_node_ids = get_all_nodes_on_street(G, 23779844, "Dennan Road")
dennan_road_nodes = {node: G.nodes[node] for node in dennan_road_node_ids if node in G.nodes}

dennan_road_markers = [
    dl.Marker(position=(dennan_road_nodes[nid]['y'], dennan_road_nodes[nid]['x']), children=dl.Tooltip(str(nid)))
    for nid in dennan_road_nodes
]


app = JupyterDash(__name__)
app.layout = html.Div([
    dl.Map(center=(centre_lat, centre_lon), zoom=18, children=[
        dl.TileLayer(),
        dl.LayerGroup(douglas_road_markers),
        dl.LayerGroup(dennan_road_markers)
    ], style={'width': '100%', 'height': '1200px'})
])

app.run(mode='inline')


/home/dave/anaconda3/envs/mapping_py12/lib/python3.12/site-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



## Creating the prompts

In [ ]:
llm_initial_prompt = """
You are a navigation assistant working with geographic road network data.

You are given:
- The current node (with latitude and longitude)
- The previous node (used to determine direction of travel)
- The name of the current road
- All nodes on that road (with latitude and longitude)
- The name of the road the user is turning onto
- All nodes on that road (with latitude and longitude)
- A turn instruction, such as "first right", "2nd left", or "exit 3rd at roundabout, which well tell you the initial direction to take from the current node"

Your job is to:
1. Determine the most likely junction node that connects the current road to the new road, based on geometry and instruction.
2. Return:
   - `junction_node`: the node ID of the junction
   - `path_to_junction`: an ordered list of node IDs starting from the current node to the junction node (inclusive)

Be precise. Use the spatial relationship between the current and previous nodes to determine forward direction, then use the turn instruction to determine which way to turn.

Always return JSON like:
```json
{
  "junction_node": 123456,
  "path_to_junction": [123123, 123124, 123456]
}

"""

-0.2938243

In [ ]:
{nid : {
        "lat": G.nodes[nid]['x'],
        "lon": G.nodes[nid]['y']}
      for nid in douglas_road_nodes
}

{23779844: {'lat': -0.2938243, 'lon': 51.381894},
 23780711: {'lat': -0.2944253, 'lon': 51.3806375},
 2578590287: {'lat': -0.2926397, 'lon': 51.3844335},
 125793172: {'lat': -0.2952753, 'lon': 51.3784392},
 134912281: {'lat': -0.2947642, 'lon': 51.3799006},
 134912282: {'lat': -0.2950307, 'lon': 51.3791745}}

In [ ]:
# Build prompt
def build_turn_prompt(
    G, current_node_id, previous_node_id, current_road_name, 
    current_road_nodes, next_road_name, next_road_nodes, instruction,
    llm_initial_prompt
):
    data = {
        "current_node": {
            "id": current_node_id,
            "lat": G.nodes[current_node_id]['x'],
            "lon": G.nodes[current_node_id]['y']
        },
        "previous_node": {
            "id": previous_node_id,
            "lat": G.nodes[previous_node_id]['x'],
            "lon": G.nodes[previous_node_id]['y']
        },
        "current_road": current_road_name,
        "current_road_nodes": [{
            "id": nid,
            "lat": G.nodes[nid]['x'],
            "lon": G.nodes[nid]['y']}
            for nid in current_road_nodes
        ],
        "next_road": next_road_name,
        "next_road_nodes": [{
            "id": nid,
            "lat": G.nodes[nid]['x'],
            "lon": G.nodes[nid]['y']}
            for nid in next_road_nodes
        ],
        "instruction": instruction
    }
    
    prompt = f"""
        {llm_initial_prompt}
        
        Here is the data:
        {json.dumps(data, indent=2)}
        
        Return JSON with:
        - junction_node (node ID)
        - path_to_junction (ordered node IDs from current node to junction)
        
        
    """
    
    return prompt.strip()

In [ ]:
## Trial prompt
doulgas_dennan_prompt = build_turn_prompt(G, 'test_centre_road', 'test_centre', 'Douglas Road', 
    get_all_nodes_on_street(G, 23779844, "Douglas Road"), 
    'Dennan Road', get_all_nodes_on_street(G, 23779844, "Dennan Road"), 
    'Right out of car park', llm_initial_prompt
)

print(doulgas_dennan_prompt)

You are a navigation assistant working with geographic road network data.

You are given:
- The current node (with latitude and longitude)
- The previous node (used to determine direction of travel)
- The name of the current road
- All nodes on that road (with latitude and longitude)
- The name of the road the user is turning onto
- All nodes on that road (with latitude and longitude)
- A turn instruction, such as "first right", "2nd left", or "exit 3rd at roundabout, which well tell you the initial direction to take from the current node"

Your job is to:
1. Determine the most likely junction node that connects the current road to the new road, based on geometry and instruction.
2. Return:
   - `junction_node`: the node ID of the junction
   - `path_to_junction`: an ordered list of node IDs starting from the current node to the junction node (inclusive)

Be precise. Use the spatial relationship between the current and previous nodes to determine forward direction, then use the turn in

In [ ]:
# Give prompt to LLM
stream = chat(
    model='qwen3:8b',
    messages=[{'role': 'user', 'content': doulgas_dennan_prompt}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

ResponseError: model requires more system memory (6.6 GiB) than is available (6.3 GiB) (status code: 500)